# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 30

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,23,23,1982,3,1,1,1,0,0,0.14,0.05,0.18,0.09,0.14,1.8,1.0,2.0,0.08,0.09,0.17,0.05,0.14,Matches
2,Max Aarons\Max-Aarons,eng ENG,DF,Norwich City,19.0,2000.0,28,28,2520,0,1,0,0,7,0,0.00,0.04,0.04,0.00,0.04,0.5,0.5,3.5,0.02,0.12,0.14,0.02,0.14,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21.0,1997.0,26,23,1943,13,3,0,0,2,0,0.60,0.14,0.74,0.60,0.74,12.3,12.3,2.3,0.57,0.11,0.68,0.57,0.68,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23.0,1996.0,23,8,740,0,2,0,0,0,0,0.00,0.24,0.24,0.00,0.24,1.9,1.9,1.0,0.23,0.12,0.35,0.23,0.35,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32.0,1987.0,11,9,875,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, engine='python')

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,30,27,2,1,66,21,45,83,57.3,30.8,26.5,0.88,W W L W D,53143,Mohamed Salah - 16,Alisson,NaN
2,Manchester City,30,20,3,7,76,31,45,63,71.5,26.4,45.2,1.51,W W L W W,54219,Sergio Agüero - 16,Ederson,NaN
3,Leicester City,30,16,6,8,59,29,30,54,48.1,36.6,11.6,0.39,D L L W D,32061,Jamie Vardy - 19,Kasper Schmeichel,NaN
4,Chelsea,30,15,6,9,53,40,13,51,51.6,29.3,22.3,0.74,L W D W W,40563,Tammy Abraham - 13,Kepa Arrizabalaga,NaN
5,Manchester Utd,30,12,10,8,45,31,14,46,45.9,29.8,16.0,0.53,W W D W D,72726,Marcus Rashford - 14,David de Gea,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,1,169,100.0,100.0,2,69140,0,0,-4,4,41.1,327,110,0,2,1.5,2.5,2,Shkodran,1.0,14,0,37.4,316,113,False,207.4,268,107,800,,2020-02-27T23:00:18.104137Z,51,0,0,0,69140.jpg,2.8,0,0,Mustafi,0.4,False,None,a,1,3,124.0,232,60,28,19566,511,41607,965,0.2,5.5,Mustafi,0
2,0,3,132,100.0,100.0,2,98745,0,0,0,0,40.9,328,111,0,2,1.5,2.5,2,Héctor,1.0,15,1,22.5,363,131,False,142.0,318,124,803,,2019-12-09T20:00:21.228098Z,55,0,0,0,98745.jpg,2.7,0,0,Bellerín,1.2,False,None,a,1,3,40.0,337,118,27,186302,1940,121984,14062,0.2,4.9,Bellerín,2
3,2,1,240,100.0,100.0,2,111457,0,0,-3,3,159.8,198,49,0,2,1.0,1.5,1,Sead,0.5,21,0,42.9,289,98,False,204.6,271,108,1193,,2020-02-23T18:30:13.672943Z,52,0,0,0,111457.jpg,2.0,0,0,Kolasinac,0.5,False,None,a,1,3,67.0,300,97,36,60493,251,130790,925,0.1,6.9,Kolasinac,3
4,2,3,218,100.0,100.0,3,154043,0,0,-5,5,157.0,201,50,1,2,1.0,1.5,1,Ainsley,0.5,20,0,45.2,277,93,False,263.4,230,92,1233,,2019-09-22T18:00:10.824841Z,45,0,0,0,154043.jpg,2.5,1,0,Maitland-Niles,2.3,False,None,a,1,3,37.0,340,120,37,606066,741,642632,2898,0.1,8.2,Maitland-Niles,3
5,0,5,305,0.0,100.0,4,39476,0,0,-2,2,36.8,337,114,1,2,0.0,0.5,0,Sokratis,0.0,25,2,58.5,224,66,False,436.2,117,44,1696,Thigh injury - Expected back 04 Jul,2020-06-18T18:00:15.974146Z,48,0,0,0,39476.jpg,3.0,0,0,Papastathopoulos,1.4,False,None,i,1,3,110.0,247,68,57,179585,105,218190,11220,0.0,11.9,Sokratis,6


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,30,27,2,1,66,21,45,83,57.3,30.8,26.5,0.88,W W L W D,53143,Mohamed Salah - 16,Alisson,NaN,1.026667,0.358199
2,Manchester City,30,20,3,7,76,31,45,63,71.5,26.4,45.2,1.51,W W L W W,54219,Sergio Agüero - 16,Ederson,NaN,0.880000,0.414783
3,Leicester City,30,16,6,8,59,29,30,54,48.1,36.6,11.6,0.39,D L L W D,32061,Jamie Vardy - 19,Kasper Schmeichel,NaN,1.220000,0.295230
4,Chelsea,30,15,6,9,53,40,13,51,51.6,29.3,22.3,0.74,L W D W W,40563,Tammy Abraham - 13,Kepa Arrizabalaga,NaN,0.976667,0.376564
5,Manchester Utd,30,12,10,8,45,31,14,46,45.9,29.8,16.0,0.53,W W D W D,72726,Marcus Rashford - 14,David de Gea,NaN,0.993333,0.370340
6,Wolves,30,11,13,6,43,34,9,46,39.6,28.2,11.4,0.38,D W W D W,31360,Raúl Jiménez - 14,Rui Patrício,NaN,0.940000,0.390628
7,Sheffield Utd,30,11,11,8,30,28,2,44,34.6,36.8,-2.2,-0.07,W D W D L,30869,Lys Mousset John Fleck - 5,Dean Henderson,NaN,1.226667,0.293269
8,Tottenham,30,11,9,10,48,41,7,42,38.8,44.3,-5.4,-0.18,W L L D D,59384,Harry Kane - 11,Paulo Gazzaniga,NaN,1.476667,0.228398
9,Crystal Palace,30,11,9,10,28,32,-4,42,28.8,38.7,-9.8,-0.33,L W W W W,25060,Jordan Ayew - 9,Vicente Guaita,NaN,1.290000,0.275271


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    if df.loc[ix, 'element_type']<=2:
        # 0.24 is the probability to miss a penalty, which incurs -2 points
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] - 6 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

In [11]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.7721357850070723

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [12]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','garcia','chicharito','elliott']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'chicharito':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('javierhernandez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [13]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [14]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
191,Salah,25.833333,186,7.2,176.015029,6.813485
618,Fernandes,6.025641,47,7.8,39.622315,6.575618
281,McGovern,2.000000,11,5.5,12.599101,6.299550
215,De Bruyne,27.971014,193,6.9,174.110159,6.224664
192,Mané,26.969697,178,6.6,160.312851,5.944184
214,Sterling,25.098039,128,5.1,143.397423,5.713491
233,Rashford,23.050847,136,5.9,128.757963,5.585823
182,Alexander-Arnold,29.827586,173,5.8,163.507140,5.481742
166,Vardy,26.825397,169,6.3,145.601556,5.427750


In [15]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [16]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total xG: ' + str(total_xG))
print('Total goals: ' + str(total_goals))
print('Total xG / total goals: ' + str(total_xG/total_goals))

Total xG: 816.7
Total goals: 787
Total xG / total goals: 1.0377382465057179
